# Neural Amp Modeler (Trainer)
This notebook allows you to train a neural amp model based on a pair of input/output WAV files that you have of the amp you want to model.

🔶**Before you run**🔶

Make sure to get a GPU! (Runtime->Change runtime type->Select "GPU" from the "Hardware accelerator dropdown menu)

⚠**Warning**⚠

Google Colab GPU instances only last for 12 hours.
Plan your training accordingly!

## Step 0: Install
Install `nam` and the other Python packages it depends on.

In [1]:
!pip install git+https://github.com/sdatkinson/neural-amp-modeler.git@issue-18

  Cloning https://github.com/sdatkinson/neural-amp-modeler.git (to revision issue-18) to /tmp/pip-req-build-8heq98wm
  Running command git clone -q https://github.com/sdatkinson/neural-amp-modeler.git /tmp/pip-req-build-8heq98wm
  Running command git checkout -b issue-18 --track origin/issue-18
  Switched to a new branch 'issue-18'
  Branch 'issue-18' set up to track remote branch 'issue-18' from 'origin'.
     |████████████████████████████████| 584 kB 4.2 MB/s 
     |████████████████████████████████| 136 kB 41.7 MB/s 
     |████████████████████████████████| 596 kB 40.0 MB/s 
     |████████████████████████████████| 409 kB 35.8 MB/s 
     |████████████████████████████████| 1.1 MB 28.7 MB/s 
     |████████████████████████████████| 144 kB 37.6 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 52.9 MB/s 
  Created wheel for nam: filename=nam-0.2.0-py3-none-any.whl size=11784 sha256=48a5ea40829ee950190cb5119225d6a3783251c8a400089239

In [2]:
from time import time
from typing import Optional, Union

import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch
from google.colab import files
from torch.utils.data import DataLoader

from nam.data import Split, init_dataset
from nam.models import Model

## Step 1: Upload files
Upload the input (DI) and output (amped) files you want to use.

The default names for the training data are `x_train.wav` (DI input) and `y_train.wav` (amped output), and for the test set, `x_test.wav` and `y_test.wav`. If you files are named differently, don't worry--you can modify the names below.

In [3]:
uploaded = files.upload()

Saving x_test.wav to x_test.wav
Saving x_train.wav to x_train.wav
Saving y_test.wav to y_test.wav
Saving y_train.wav to y_train.wav


## Step 2: Settings
The defaults are what I tend to start with and should usually work well (except the file names--see above), but if you'd like, you can make changes.

In [28]:
data_config = {
    "train": {
        "x_path": "x_train.wav",
        "y_path": "y_train.wav",
        "ny": 1024
    },
    "validation": {
        "x_path": "x_test.wav",
        "y_path": "y_test.wav",
        "ny": None
    },
    "common": {
        "nx": 8191
    }
}
model_config = {
    "net": {
        "name": "ConvNet",
        "config": {
            "channels": 16,
            "dilations": [
                1,
                2,
                4,
                8,
                16,
                32,
                64,
                128,
                256,
                512,
                1024,
                2048,
                1,
                2,
                4,
                8,
                16,
                32,
                64,
                128,
                256,
                512,
                1024,
                2048
            ],
            "batchnorm": True,
            "activation": "Tanh",
        }
    },
    "optimizer": {
        "lr": 0.003
    },
    "lr_scheduler": {
        "class": "ReduceLROnPlateau",
        "kwargs": {
            "factor": 0.7,
            "patience": 20,
            "cooldown": 20,
            "min_lr": 1e-05,
            "verbose": True
        },
        "monitor": "val_loss"
    }
}
learning_config = {
    "train_dataloader": {
        "batch_size": 16,
        "shuffle": True,
        "pin_memory": True,
        "drop_last": True,
        "num_workers": 4
    },
    "val_dataloader": {
    },
    "trainer": {
        "gpus": 1,
        "max_epochs": 100
    },
    "trainer_fit_kwargs": {
    }
}

## Step 3: Run!
Let's rock

In [29]:
dataset_train = init_dataset(data_config, Split.TRAIN)
dataset_validation = init_dataset(data_config, Split.VALIDATION)
train_dataloader = DataLoader(dataset_train, **learning_config["train_dataloader"])
val_dataloader = DataLoader(dataset_validation, **learning_config["val_dataloader"])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [30]:
model = Model.init_from_config(model_config)

In [33]:
trainer = pl.Trainer(
    callbacks=[
        pl.callbacks.model_checkpoint.ModelCheckpoint(
            filename="{epoch}_{val_loss:.6f}",
            save_top_k=3,
            monitor="val_loss",
            every_n_epochs=1,
        ),
        pl.callbacks.model_checkpoint.ModelCheckpoint(
            filename="checkpoint_last_{epoch:04d}", every_n_epochs=1
        ),
    ],
    **learning_config["trainer"],
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# Here we go!
trainer.fit(
    model,
    train_dataloader,
    val_dataloader,
    **learning_config.get("trainer_fit_kwargs", {}),
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | _net | ConvNet | 12.6 K
---------------------------------
12.6 K    Trainable params
0         Non-trainable params
12.6 K    Total params
0.050     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
# Go to best checkpoint
best_checkpoint = trainer.checkpoint_callback.best_model_path
if best_checkpoint != "":
    model = Model.load_from_checkpoint(
        trainer.checkpoint_callback.best_model_path,
        **Model.parse_config(model_config),
    )
model.eval()

# Step 4: Check
Let's look at how good our model matches the real thing

In [ ]:
def _rms(x: Union[np.ndarray, torch.Tensor]) -> float:
    if isinstance(x, np.ndarray):
        return np.sqrt(np.mean(np.square(x)))
    elif isinstance(x, torch.Tensor):
        return torch.sqrt(torch.mean(torch.square(x))).item()
    else:
        raise TypeError(type(x))

def plot(
    model,
    ds,
    savefig=None,
    show=True,
    window_start: Optional[int] = None,
    window_end: Optional[int] = None,
):
    with torch.no_grad():
        tx = len(ds.x) / 48_000
        print(f"Run (t={tx})")
        t0 = time()
        output = model(ds.x).flatten().cpu().numpy()
        t1 = time()
        print(f"Took {t1 - t0} ({tx / (t1 - t0):.2f}x)")

    plt.figure(figsize=(16, 5))
    # plt.plot(ds.x[window_start:window_end], label="Input")
    plt.plot(output[window_start:window_end], label="Prediction")
    plt.plot(ds.y[window_start:window_end], linestyle="--", label="Target")
    # plt.plot(
    #     ds.y[window_start:window_end] - output[window_start:window_end], label="Error"
    # )
    plt.title(f"NRMSE={100.0 * _rms(torch.Tensor(output) - ds.y) / _rms(ds.y):2.1f}\%")
    plt.legend()
    if savefig is not None:
        plt.savefig(savefig)
    if show:
        plt.show()

In [ ]:
plot(
    model,
    dataset_validation,
    window_start=100_000,  # Start of the plotting window, in samples
    window_end=110_000,  # End of the plotting window, in samples
)

## Step 5: Export your model
Now we'll use NAM's exporting utility to convert the model from its PyTorch representation to something that you can put into the plugin.

In [12]:
# TODO

## Step 6: Download your artifacts
We're done! 
Go to the file browser on the left panel and collect your artifacts!

Be sure to download the lightning model artifacts (in case you want to continue training later) and your exported model (so that you can put it into a plugin).